#### Librerias

In [ ]:
import random
from timeit import default_timer as timer
import minizinc

#### Hallar la ubicación del MiniZinc

In [ ]:
my_driver = minizinc.find_driver(["C:/Program Files/MiniZinc"])
my_driver.make_default()

#### MiniZinc funciones

In [ ]:
from minizinc import Instance, Model, Solver

#### Funcion que retorna en un arreglo la solución factible de MiniZinc

In [ ]:
def satisfymini(archivo):
    gecode = Solver.lookup("gecode")

    trivial = minizinc.Model([archivo])
    instance = Instance(gecode, trivial)
    # Find and print all intermediate solutions
    result = instance.solve()
    s=[]
    for i in range(1,n+1):
        variable="x" + str(i)
        s.append(int(result[variable]))
    return(s)

#### Proyecto 2 - Generador de instancias modificado

In [ ]:
for t in range (0,1):
    n= 50 #cantidad de variable
    mv=4 #cantidad maxima de vecinos 
    mn=1 #cantidad minima de vecionos
    z=random.randint(1,int(n/3))    
    ar="datos"+str(t+1)+".lp"
    ar2="datos2"+str(t+1)+".mzn"
    ar3="restriccion.txt"
    archi1=open(ar,"w")
    archi2=open(ar2,"w")
    archi3=open(ar3,"w")
    archi1.write("min: ") 
    for i in range (0,n):
        archi2.write("var 0..1: x"+str(i+1)+";\n")
    archi2.write("var int: ct;\n\n")
    archi2.write("constraint ct= ")
    for i in range (0,n):
        archi1.write("x"+str(i+1))
        archi2.write("1*x"+str(i+1))
        if i+1<n:
            archi1.write(" + ")
            archi2.write(" + ")
        elif i+1==n:
            archi1.write(";")
            archi2.write(";")
    archi1.write("\n")
    archi2.write("\n\n")
    for i in range (1,z+1):
        v=[]
        archi2.write("constraint ")
        for j in range(random.randint(mn,mv),0,-1):
            flag=0
            while flag==0:
                s=random.randint(1,n)
                if s not in v:
                    v.append(s)
                    flag=1
                    if j==1:
                        archi1.write("x"+str(s)+" = 1;\n")
                        archi2.write("x"+str(s)+" = 1;\n")
                        archi3.write(str(s)+"\n")
                    else:
                        archi1.write("x"+str(s)+" + ")
                        archi2.write("x"+str(s)+" + ")
                        archi3.write(str(s)+",")
    archi1.write("bin ")
    archi2.write("\nsolve satisfy;")
    for i in range (0,n):
        archi1.write("x"+str(i+1))
        if i+1<n:
            archi1.write(", ")
        elif i+1==n:
            archi1.write(";")
    archi1.close()
    archi2.close()
    archi3.close()

#### Función que crea una lista con las restricciones 

In [ ]:
def crear_lista_restricciones(archivo):
    lista=[]
    arch=open(archivo,"r")
    while arch:
        linea=arch.readline()
        if linea =="":
            break
        x=linea.strip().split(",")
        y=list(map(int,x))
        z=sorted(y)
        lista.append(z)
    arch.close()
    return(lista)

#### Funciones auxiliares para ejecutar Hill Climbing

In [ ]:
def change(v,i):
    v[i]=(v[i]+1)%2

def restriccion(r,v): # x11+x2+x4=1    [[11,2,4],[]] r=[], v=[]
    sum=0
    for i in r:
        sum=sum+v[i-1]
    return sum

#### Complemento simple

In [ ]:
def complsimple(s,v):
    for i in range(0,len(s)):
        c=list(s)
        change(c,i)
        v.append(list(c))

#### Hill Climbing 1 - la primera que mejora, complemento simple

In [ ]:
def hillclimbing(s,r,n,best):
    flag=0  
    a=0
    best=[s,sum(s)]
    flag2=1                      
    while flag==0 and a<n:
        v=[]
        flag=1
        sa=list(best[0])
        complsimple(best[0],v)
        for j in v:
            flag2=1
            for i in r:
                  if restriccion(i,j)!=1:
                        flag2=0
            if flag2==1:
                if sum(j)<best[1]:
                    best=[j,sum(j)]
                    flag=0
                    break 
        a=a+1
    return(best)

#### Hill Climbing 2 - Solucion que mas mejora, complemento simple

In [ ]:
def hillclimbing_2(s,r,n,best):    
    flag=0
    a=0
    best=[s,sum(s)]
    flag2=1
    while flag==0 and a<n:
        v=[]
        flag=1
        complsimple(best[0],v)
        for j in v:
            flag2=1
            for i in r:
                  if restriccion(i,j)!=1:
                        flag2=0
            if flag2==1:
                if sum(j)<best[1]:
                    best=[j,sum(j)]
                    flag=0
        a=a+1
    return(best)

#### Complemento doble

In [ ]:
def compldoble(s,vd):
    for i in range(0,len(s)):
            for j in range (i+1,len(s)):
                    c=list(s)
                    change(c,i)
                    change(c,j)
                    vd.append(c)
    complsimple(s,vd)

#### Hill Climbing 3 - Primera que mejora, complemento doble

In [ ]:
def hillclimbing_3(s,r,n,best):  
    flag=0
    a=0
    best=[s,sum(s)]                   
    flag2=1
    while flag==0 and a<n:
        v=[]
        flag=1
        compldoble(best[0],v)
        for j in v:
            flag2=1
            for i in r:
                  if restriccion(i,j)!=1:
                        flag2=0
            if flag2==1:
                if sum(j)<best[1]:
                    best=[j,sum(j)]
                    flag=0
                    break 
        a=a+1
    return(best)

#### Hill Climbing 4 - La que más mejora, complemento doble

In [ ]:
def hillclimbing_4(s,r,n,best):
    flag=0
    a=0
    flag2=1
    for i in r:
           if restriccion(i,s)!=1:
                flag2=0
    if flag2==1:
            best=[s,sum(s)]
    else:
            best=[s,sum(s)*10000000]
    flag2=1
    while flag==0 and a<n: #la que más mejora, complemento doble
        v=[]
        flag=1
        compldoble(best[0],v)
        for j in v:
            flag2=1
            for i in r:
                  if restriccion(i,j)!=1:
                        flag2=0
            if flag2==1:
                if sum(j)<best[1]:
                    best=[list(j),sum(j)]
                    flag=0
        a=a+1
    return(best)

#### Tiempo de ejecución Hill Climbing 1, 2, 3 y 4.

In [ ]:
def medirtiempo(s,r,n,lista):
    tiempos=[]
    resultados=[]
    start=timer()
    a=hillclimbing(s,r,n/2,lista)
    end=timer()
    start1=timer()
    b=hillclimbing_2(s,r,n/2,lista)
    end1=timer()
    start2=timer()
    c=hillclimbing_3(s,r,n/2,lista)
    end2=timer()
    start3=timer()
    d=hillclimbing_4(s,r,n/2,lista)
    end3=timer()
    resultados.append(a[1])
    resultados.append(b[1])
    resultados.append(c[1])
    resultados.append(d[1])
    tiempos.append(float(str(end-start)))
    tiempos.append(float(str(end1-start1)))
    tiempos.append(float(str(end2-start2)))
    tiempos.append(float(str(end3-start3)))
    return tiempos, resultados

#### Permutar solución factible.

In [ ]:
def permutardatos(s,r,datos):
    for b in range(0,int(datos/3)):
        a=random.randint(0,datos)
        flag=1
        for i in r:
            if a+1 in i:
                flag=0
                break
        if flag==1 and a!=datos:
            s[a]=1
    return(sum(s))